In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# ==================================
# S1: 数据信息和预处理
# ==================================
try:
    df = pd.read_csv('subdata.csv')
    print("数据集加载成功。")
    print("数据集形状:", df.shape)
except FileNotFoundError:
    print("错误: subdata.csv 文件未找到。请确保文件在正确的目录中。")
    exit()

# 显示数据集基本信息
print("\n数据集信息:")
df.info()

print("\n数据集描述性统计:")
print(df.describe())

print(f"\n目标变量 'TEACHBEHA' 的缺失值数量: {df['TEACHBEHA'].isnull().sum()}")

# 移除 'TEACHBEHA' 为空的行
df.dropna(subset=['TEACHBEHA'], inplace=True)
print(f"'TEACHBEHA' 缺失值处理后形状: {df.shape}")

# 确定特征 (X) 和目标 (y)
y = df['TEACHBEHA']
X = df[["STUBEHA"]]
# X = df.drop('TEACHBEHA', axis=1)

# ==================================
# S2: 数据集划分
# ==================================

# 使用固定比例划分训练集和测试集 (80% 训练, 20% 测试)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# random_state表示随机种
print(f"训练集大小: {X_train.shape[0]} ({len(X_train)/len(X)*100:.2f}%)")
print(f"测试集大小: {X_test.shape[0]} ({len(X_test)/len(X)*100:.2f}%)")

# ==================================
# S4: 特征选择与处理
# ==================================

# 识别数值和分类特征
# 将二者分离以进一步分析
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
# categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()


print(f"\n数值特征 ({len(numeric_features)}): {numeric_features}")
# print(f"分类特征 ({len(categorical_features)}): {categorical_features}")

# 创建预处理管道
# 数值特征: 缺失值填充 (使用中位数) + 标准化缩放
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# 分类特征: 缺失值填充 (使用 'missing' 策略) + One-Hot 编码
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# 创建一个 ColumnTransformer 来应用不同的转换
# 分别应对数值特征和非数值特征
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
        ])

#('cat', categorical_transformer, categorical_features)

# ==================================
# S5: 主实验
# ==================================

# 定义评估函数
def evaluate_model(name, model, X_test_processed, y_test_true):
    """评估模型并打印结果。"""
    y_pred = model.predict(X_test_processed)
    mse = mean_squared_error(y_test_true, y_pred)
    r2 = r2_score(y_test_true, y_pred)
    print(f"\n--- {name} 评估结果 ---")
    print(f"均方误差 (MSE): {mse:.4f}")
    print(f"R² 分数: {r2:.4f}")
    return mse, r2

# 创建包含预处理和模型的完整管道
# 模型 1: 线性回归
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', LinearRegression())])

# 模型 2: 随机森林回归
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', RandomForestRegressor(random_state=42))])

# 训练模型
print("\n训练线性回归模型...")
lr_pipeline.fit(X_train, y_train)
print("训练完成。")

print("\n训练随机森林回归模型...")
rf_pipeline.fit(X_train, y_train)
print("训练完成。")

# 在测试集上评估模型
lr_mse, lr_r2 = evaluate_model("线性回归", lr_pipeline, X_test, y_test)
rf_mse, rf_r2 = evaluate_model("随机森林回归", rf_pipeline, X_test, y_test)

# 可视化比较
results_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'MSE': [lr_mse, rf_mse],
    'R2': [lr_r2, rf_r2]
})

print("\n--- 结果汇总 ---")
print(results_df)

# ==================================
# S6: 参数实验 (以随机森林为例)
# ==================================
print("我们将使用 GridSearchCV 对随机森林的 n_estimators 参数进行调优。")
print("注意: 这可能需要一些时间...")

# 定义参数网格
param_grid = {
    'regressor__n_estimators': [50, 100, 200],  # 尝试不同数量的树
    'regressor__max_depth': [None, 10, 20]      # 尝试不同最大深度
}

# 创建包含预处理和随机森林的管道
rf_grid_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('regressor', RandomForestRegressor(random_state=42))])

# 使用 3 折交叉验证进行网格搜索
grid_search = GridSearchCV(rf_grid_pipeline, param_grid, cv=3,
                           scoring='neg_mean_squared_error', # 使用负 MSE 作为评分
                           verbose=1, n_jobs=-1) # 使用所有可用的 CPU 核心

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 获取最佳参数和最佳模型
print("\n最佳参数:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_

# 使用最佳模型进行评估
best_rf_mse, best_rf_r2 = evaluate_model("随机森林 (调优后)", best_rf_model, X_test, y_test)

print(f"\n调优前 MSE: {rf_mse:.6f}, R2: {rf_r2:.6f}")
print(f"调优后 MSE: {best_rf_mse:.6f}, R2: {best_rf_r2:.6f}")


数据集加载成功。
数据集形状: (1089, 26)

数据集信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1089 non-null   int64  
 1   CNTSCHID          1089 non-null   int64  
 2   Region            1089 non-null   int64  
 3   STRATUM           1089 non-null   object 
 4   LANGTEST          1064 non-null   float64
 5   PRIVATESCH        1089 non-null   object 
 6   SCHLTYPE          1040 non-null   float64
 7   STRATIO           977 non-null    float64
 8   SCHSIZE           988 non-null    float64
 9   RATCMP1           970 non-null    float64
 10  RATCMP2           1018 non-null   float64
 11  TOTAT             1024 non-null   float64
 12  PROATCE           894 non-null    float64
 13  PROAT5AB          857 non-null    float64
 14  PROAT5AM          790 non-null    float64
 15  PROAT6            870 non-null    float64
 16  CLSIZE 

In [15]:
from openai import OpenAI
from key import API_KEY

client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "已知给定pandas.DataFrame实例df，请编写一段Python代码，分别求特征STUBEHA与TEACHBA、EDUSHORT与STAFFSHORT的相关系数。请逐步思考输出答案。"},
        {"role": "assistant", 
         "content": "以下是一个类似的示例代码，用于计算相关系数并绘制热力图："
         "\n\n```python"
         "\nimport pandas as pd"
         "\nimport numpy as np"
         "\nimport matplotlib.pyplot as plt"
         "\n\n# 假设df是你的DataFrame"
         "\n# 计算相关系数"
         "\ncorrelation = df[['STUBEHA', 'TEACHBA', 'EDUSHORT', 'STAFFSHORT']].corr()"
         "\n\n# 将相关系数矩阵转换为numpy数组"
         "\ncor_npy = correlation.to_numpy()"
         "\n\n# 绘制热力图"
         "\nplt.clf()"
         "\nplt.title('Correlation of selected features')"
         "\nplt.xticks(ticks=[0, 1, 2, 3], labels=correlation.columns, rotation=45, rotation_mode='anchor', ha='right')"
         "\nplt.yticks(ticks=[0, 1, 2, 3], labels=correlation.columns)"
         "\nfor i in range(4):"
         "\n    for j in range(4):"
         "\n        plt.text(j, i, f'{cor_npy[i, j]:.3f}', va='center', ha='center')"
         "\nplt.imshow(correlation, vmin=-1, vmax=1)"
         "\nplt.colorbar()"
         "\nplt.show()"
         "\n```\n请根据你的需求调整上述代码。"},
        {"role": "user", "content": "已知给定pandas.DataFrame实例df，请编写一段Python代码，分别求特征STUBEHA与TEACHBA、EDUSHORT与STAFFSHORT的相关系数。请逐步思考输出答案。"},
    ],
    stream=False
)

print(response.choices[0].message.content)

以下是分别计算两对特征相关系数的Python代码实现：

```python
# 分别计算两组特征的相关系数
corr_stu_teach = df['STUBEHA'].corr(df['TEACHBA'])
corr_edu_staff = df['EDUSHORT'].corr(df['STAFFSHORT'])

# 打印结果（保留三位小数）
print(f"STUBEHA与TEACHBA的相关系数: {corr_stu_teach:.3f}")
print(f"EDUSHORT与STAFFSHORT的相关系数: {corr_edu_staff:.3f}")
```

**代码解析：**

1. **直接计算法**：使用`pd.Series.corr()`方法直接计算两个Series之间的Pearson相关系数
   - `df['STUBEHA']` 选取STUBEHA特征列
   - `.corr(df['TEACHBA'])` 计算其与TEACHBA列的相关系数

2. **高效计算**：相比先计算完整相关系数矩阵的方法，直接计算法：
   - 避免计算不必要的特征组合
   - 节省内存（不生成中间矩阵）
   - 时间复杂从O(n²)降为O(n)

3. **结果格式化**：使用f-string格式化输出结果，保留3位小数

**扩展说明：**
- 若需要同时获取多组相关系数，可以使用矩阵法：
```python
corr_matrix = df[['STUBEHA','TEACHBA','EDUSHORT','STAFFSHORT']].corr()
# 提取特定组合
print(corr_matrix.loc['STUBEHA', 'TEACHBA'])
print(corr_matrix.loc['EDUSHORT', 'STAFFSHORT'])
```
- 支持其他相关系数类型（如Spearman）：
```python
df['STUBEHA'].corr(df['TEACHBA'], method='spearman')
```
